#Libraries Importation and Installation

In [0]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import spacy
from spacy.lang.en import English
from sklearn.ensemble import RandomForestClassifier
from collections import Counter 
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import re
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegressionCV
RSEED = 42

In [0]:
#!pip install spacy
#!apt install -qq enchant
#!pip install pyenchant
#!python -m spacy download en

#Import Data

In [0]:
data = pd.read_csv('https://raw.githubusercontent.com/XaviJunior/SBB/master/project_2/Data/train.csv',encoding="utf8")
y=data['target']
X=data['text']

In [0]:
data_test=pd.read_csv('https://raw.githubusercontent.com/XaviJunior/SBB/master/project_2/Data/test.csv',encoding='utf8')
Text_test=data_test['text']

#Data Cleaning

In [0]:
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

#TFIDF-Vectorizer


In [0]:
vect=TfidfVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))
Text_test=vect.fit_transform(Text_test)
X=vect.transform(X)

#Train-Test-Split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RSEED)

ML

In [217]:
RF = RandomForestClassifier(criterion="entropy", n_estimators=51, max_depth=306, random_state=RSEED)
RF.fit(X_train,y_train)
RF.score(X_test,y_test)

0.7715036112934996

#Prediction Function

In [0]:
def prediction(test,algo):
  algo.fit(X,y)
  pred=algo.predict(test)
  data_test['target']=pred
  Fin=data_test[['id','target']]
  from google.colab import drive
  drive.mount('drive')
  Fin.to_csv('UNIL_SBB.csv',index=False)
  !cp UNIL_SBB_1st.csv "drive/My Drive/"

First Prediction


In [25]:
prediction(Text_test,RF)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive
cp: cannot stat 'UNIL_SBB_1st.csv': No such file or directory


Result = 0.77811
Rank=1763

#Exchange fit_transform

In [0]:
X=data['text']
Text_test=data_test['text']

In [0]:
vect=TfidfVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))
X=vect.fit_transform(X)
Text_test=vect.transform(Text_test)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RSEED)

In [222]:
RF_2 = RandomForestClassifier(criterion="entropy", n_estimators=51, max_depth=306, random_state=RSEED)
RF_2.fit(X_train,y_train)
RF_2.score(X_test,y_test)

0.7741300065659882

##Prediction

In [0]:
prediction(Text_test,RF_2)

0.79447

Naive-Bayes

In [226]:
clf_bern=BernoulliNB()
clf_multi=MultinomialNB()
clf_bern.fit(X_train,y_train)
clf_multi.fit(X_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [227]:
print("Bernoulli ",clf_bern.score(X_test,y_test))
print("Multi ",clf_multi.score(X_test,y_test))

Bernoulli  0.7964543663821405
Multi  0.7898883782009193


Extra Cleaning

In [0]:
df = pd.read_csv("https://raw.githubusercontent.com/XaviJunior/SBB/master/project_2/Data/train.csv", encoding="utf-8")

In [0]:
df_test = pd.read_csv("https://raw.githubusercontent.com/XaviJunior/SBB/master/project_2/Data/test.csv", encoding="utf-8")

In [0]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [0]:
def clean(tweet):
    #remove url
    tweet = re.sub(r"http\S{0,}", " ", tweet)
    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)

    #tweet = re.sub(r".Û.", "", tweet)
    #tweet = re.sub(r"\s+", " ", tweet)
    #tweet = re.sub(r'\t', " ", tweet)
    #tweet = re.sub(r'\n', " ", tweet)
    #tweet = re.sub(r"\.{3,3}", "", tweet)
   
    #Replace special characters with a space
    special="#!$%&'*+,-./;<=>?@[]^_`{|}~"
    for i in special:
      tweet=tweet.replace(i," ")
    
     # tweet=re.sub(r'[^\w]', ' ', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)

    
    #remove tokens with number only
    tweet=re.sub(r'\b[0-9]+\b\s*', '', tweet)


    return tweet
    
df["text"] = df["text"].apply(clean)
df_test["text"] = df_test["text"].apply(clean)


In [0]:
nlp = English()
punctuation = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def tokenizer(tweet):
    tokens = nlp(tweet)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens = [word for word in tokens if word not in stop_words and word not in punctuation]
    
    return tokens

# vectorizer = TfidfVectorizer(tokenizer=tokenizer, ngram_range=(1,1))
vectorizer = CountVectorizer(tokenizer=tokenizer, ngram_range=(1,1))

In [417]:
X = vectorizer.fit_transform(df["text"].values.tolist())
y = df["target"].values.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RSEED)

['(:', '):', '00end', '00pm', '02pm', '05th', '06jst', '0l', '0npzp', '1', '10', '100', '1000s', '1008pla', '1008planet', '100bn', '100nd', '100s', '1061thetwister', '10k', '10news', '10th', '10w', '10x', '11', '115film', '12', '12hr', '12jst', '12news', '12th', '12u', '12v', '13pm', '14th', '15', '15moferadio', '15p', '15pm', '15th', '16yr', '17months', '17th', '18jst', '18th', '18w', '1960s', '1970s', '19th', '19yrs', '1a', '1acd4900c1424d1', '1d', '1pack', '1rockstar62', '1st', '1x1', '2', '20000k', '2007he', '203rd', '20k', '20mins', '20s', '20skyhawkmm20', '20yrs', '21k', '21st', '22days', '22pm', '23', '231a', '233liveonline', '23rd', '24v', '2500fps', '250k', '260th', '263chat', '265v', '27w', '28pm', '295ss', '29pm', '2a', '2b', '2d', '2day', '2fast2furious', '2for1lapdances', '2hours', '2k13', '2k15', '2leezy', '2minutemix', '2nd', '2pack', '2pcs', '2slow2report', '3', '30', '300', '300k', '300w', '301dc', '30a', '30bst', '30min', '30p', '30pcs', '30pm', '30s', '30stm', '30th'

In [412]:
clf_bern.fit(X_train,y_train)
clf_multi.fit(X_train,y_train)
print("Bernoulli ",clf_bern.score(X_test,y_test))
print("Multi ",clf_multi.score(X_test,y_test))

Bernoulli  0.7984241628365069
Multi  0.7872619829284307


In [418]:
LR = LogisticRegressionCV(solver="lbfgs", cv=5, max_iter=1000, random_state=RSEED)
LR.fit(X_train, y_train)
LR.score(X_test,y_test)

0.8049901510177282

In [0]:
Text_test = vectorizer.transform(df_test["text"].values.tolist())

In [0]:
prediction(Text_test,LR)

#Nu-Support Vector Classification

In [429]:
from sklearn import svm
clf = svm.NuSVC()
clf.fit(X_train,y_train)
print(clf.score(X_test,y_test))

0.8010505581089954


In [431]:
prediction(Text_test,clf)

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
cp: cannot stat 'UNIL_SBB_1st.csv': No such file or directory
